In [1]:
# Tensorflow uses CPU instead of GPU device
import os
%env CUDA_VISIBLE_DEVICES=0
%env KERAS_BACKEND=tensorflow

env: CUDA_VISIBLE_DEVICES=0
env: KERAS_BACKEND=tensorflow


In [2]:
import pandas as pd

from SMILESX import main, inference
%matplotlib inline

Using TensorFlow backend.




Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



RDKit WARNING: [16:09:14] Enabling RDKit 2019.09.3 jupyter extensions


#### Definition of data

In [3]:
validation_data_dir = "validation_data/"

In [4]:
extension = '.csv'

In [5]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [6]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [7]:
import os
try:
    data_file = os.path.join(validation_data_dir,data_filename+extension)
    sol_data = pd.read_csv(os.path.join(data_file))
except FileNotFoundError:
    print(f"file {data_file} doesn't exist")

In [8]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


In [9]:
sol_data = sol_data[['smiles',prop_tag]]

In [10]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [11]:
sol_data.shape

(642, 2)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [12]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 
dalpha_range = [float(ialpha/10.) for ialpha in range(20,40,1)] # Adam's learning rate = 10^(-dalpha_range)

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'lrate', 'type': 'discrete', 'domain': dalpha_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': (1024, 1024)}, # fixed
        {'name': 'lrate', 'type': 'discrete', 'domain': (3, 3)} # fixed
    ]

In [ ]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 3, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_epochs = 1,
          bayopt_n_rounds = 1, 
          bayopt_it_factor = 1, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          n_gpus = 1, 
          bridge_type = 'NVLink', 
          batch_size_ref = 8,
          alpha_ref = 22, 
          patience = 50, 
          n_epochs = 100)

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Sampling and splitting of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.80/0.10/0.10


***Data augmentation to True***

Enumerated SMILES:
	Training set: 4352
	Validation set: 474
	Test set: 529

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', '=', 'C', 'C', 'C', ' '], [' ', 'C', '(', 'C', 'C', ')', '=', 'C', ' '], [' ', 'C', '(', 'C', ')', 'C', '=', 'C', ' '], [' ', 'C', 'C', 'C', '=', 'C', ' '], [' ', 'c', '1', 'c', 'c', 'c', '2', 'n', 'c', 'c', 'c', 'c', '2', 'c', '1', ' ']]

Number of tokens only present in a training set: 33

Number of tokens only present in a validation set: 22
Is the validation set a subset of the training set: True
What are the tokens by which they differ: set()

Number of tokens only present in a test set: 26
Is the test set a subset of 

17/17 [==============================] - 27s 2s/step - loss: 0.0235 - mean_absolute_error: 0.1200 - mean_squared_error: 0.0235 - val_loss: 0.0298 - val_mean_absolute_error: 0.1379 - val_mean_squared_error: 0.0298
Epoch 9/100
17/17 [==============================] - 27s 2s/step - loss: 0.0245 - mean_absolute_error: 0.1200 - mean_squared_error: 0.0245 - val_loss: 0.0268 - val_mean_absolute_error: 0.1346 - val_mean_squared_error: 0.0268
Epoch 10/100
17/17 [==============================] - 26s 2s/step - loss: 0.0224 - mean_absolute_error: 0.1192 - mean_squared_error: 0.0224 - val_loss: 0.0279 - val_mean_absolute_error: 0.1409 - val_mean_squared_error: 0.0279
Epoch 11/100
17/17 [==============================] - 27s 2s/step - loss: 0.0219 - mean_absolute_error: 0.1170 - mean_squared_error: 0.0219 - val_loss: 0.0268 - val_mean_absolute_error: 0.1319 - val_mean_squared_error: 0.0268
Epoch 12/100
17/17 [==============================] - 27s 2s/step - loss: 0.0243 - mean_absolute_error: 0.1209

17/17 [==============================] - 26s 2s/step - loss: 0.0112 - mean_absolute_error: 0.0811 - mean_squared_error: 0.0112 - val_loss: 0.0094 - val_mean_absolute_error: 0.0746 - val_mean_squared_error: 0.0094
Epoch 45/100
17/17 [==============================] - 27s 2s/step - loss: 0.0103 - mean_absolute_error: 0.0779 - mean_squared_error: 0.0103 - val_loss: 0.0091 - val_mean_absolute_error: 0.0729 - val_mean_squared_error: 0.0091
Epoch 46/100
17/17 [==============================] - 26s 2s/step - loss: 0.0102 - mean_absolute_error: 0.0765 - mean_squared_error: 0.0102 - val_loss: 0.0127 - val_mean_absolute_error: 0.0851 - val_mean_squared_error: 0.0127
Epoch 47/100
17/17 [==============================] - 27s 2s/step - loss: 0.0105 - mean_absolute_error: 0.0790 - mean_squared_error: 0.0105 - val_loss: 0.0137 - val_mean_absolute_error: 0.0901 - val_mean_squared_error: 0.0137
Epoch 48/100
17/17 [==============================] - 26s 2s/step - loss: 0.0094 - mean_absolute_error: 0.074

In [ ]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

In [21]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.377222,0.0105852
1,CCC,0.398645,0.00856436
2,C=O,-0.0439493,0.0381139
